In [1]:
import pandas as pd
import numpy as np
import functools as ft

In [2]:
def identify_reservations(df):
    try:
        Fab300_raw_reservations = df.copy()
        Fab300_raw_reservations["DATE_TIME_STAMP"] = pd.to_datetime(Fab300_raw_reservations["DATE_TIME_STAMP"])
        SortedRows = Fab300_raw_reservations.sort_values(["EVENT_ROW_ID"])

        index = range(1,len(SortedRows)+1)
        IndexShift_1 = [i-1 if i%2 != 0 else np.nan for i in index]
        IndexShift_2 = [i-1 if i%2 == 0 else np.nan for i in index]

        SortedRows["Index"] = index
        SortedRows["IndexShift_1"] = IndexShift_1
        SortedRows["IndexShift_2"] = IndexShift_2
        SortedRows["IndexShift_1_1"] = SortedRows["IndexShift_1"].fillna(method='bfill')
        SortedRows["IndexShift_2_1"] = SortedRows["IndexShift_2"].fillna(method='bfill')
        SortedRows = SortedRows.drop(columns=['IndexShift_1', 'IndexShift_2'])
        SortedRows = SortedRows.rename(columns={"IndexShift_1_1":"IndexShift_1","IndexShift_2_1":"IndexShift_2"})
        FilledUp2 = SortedRows.copy()

        UnpivotedOnlySelectedColumns = pd.melt(FilledUp2, id_vars=['FO_ROW_ID','Index','IndexShift_1','IndexShift_2'], 
                    value_vars=["ResWBS", "ResTk", "DATE_TIME_STAMP", "USER_ID", "EVENT_ROW_ID"])

        UnpivotedOnlySelectedColumns["ResProperty_1"] = np.where(UnpivotedOnlySelectedColumns['Index']==UnpivotedOnlySelectedColumns['IndexShift_1'], UnpivotedOnlySelectedColumns['variable'] + "_Start", UnpivotedOnlySelectedColumns['variable'] + "_End")
        UnpivotedOnlySelectedColumns["ResProperty_2"] = np.where(UnpivotedOnlySelectedColumns['Index']==UnpivotedOnlySelectedColumns['IndexShift_2'], UnpivotedOnlySelectedColumns['variable'] + "_Start", UnpivotedOnlySelectedColumns['variable'] + "_End")

        AddedCustom4 = UnpivotedOnlySelectedColumns.copy()
        RemovedColumns1 = AddedCustom4.drop(columns=["Index", "IndexShift_2", "variable", "ResProperty_2"])

        PivotedColumn1 = RemovedColumns1.pivot(index=['FO_ROW_ID','IndexShift_1'],columns='ResProperty_1',values='value').reset_index()

        FilteredRows01 = PivotedColumn1[(PivotedColumn1["ResWBS_Start"].notnull() & PivotedColumn1['ResWBS_Start'].str.len() > 0)]

        RemovedColumns2 = AddedCustom4.drop(columns=["Index", "IndexShift_1", "variable", "ResProperty_1"])
        PivotedColumn2 = RemovedColumns2.pivot(index=['FO_ROW_ID','IndexShift_2'],columns='ResProperty_2',values='value').reset_index()

        FilteredRows02 = PivotedColumn2[(PivotedColumn2["ResWBS_Start"].notnull() & PivotedColumn2['ResWBS_Start'].str.len() > 0)]

        columns = [
            'FO_ROW_ID', 
            'DATE_TIME_STAMP_End',
            'DATE_TIME_STAMP_Start', 
            'EVENT_ROW_ID_End', 
            'EVENT_ROW_ID_Start',
            'ResTk_End', 
            'ResTk_Start', 
            'ResWBS_End', 
            'ResWBS_Start', 
            'USER_ID_End',
            'USER_ID_Start'
        ]
        FilteredRows01 = FilteredRows01[columns]
        FilteredRows02 = FilteredRows01[columns]
        combined = pd.concat([FilteredRows01,FilteredRows02])

        combined = combined.rename(
            columns=
            {
                "ResTk_Start":"ResTk",
                "ResWBS_Start":"WBS",
                "EVENT_ROW_ID_Start": "EVENT_ROW_ID_Begin",
                "DATE_TIME_STAMP_Start": "DATE_TIME_STAMP_Begin"
            }
        )

        columns = [
            "FO_ROW_ID",
            "ResTk", 
            "WBS", 
            "EVENT_ROW_ID_Begin", 
            "EVENT_ROW_ID_End", 
            "DATE_TIME_STAMP_Begin", 
            "DATE_TIME_STAMP_End", 
            "USER_ID_Start", 
            "USER_ID_End"
        ]
        Tools_with_resersvations  = combined#[columns]

        final = combined[columns]
        return final
    except:
        return pd.DataFrame()

In [4]:
Fab300_raw_reservations = pd.read_csv("C:\\Users\\fpicaso\\Repos\\Teo-2022-10\\20221108Fab300_raw_reservations.csv")
Fab300_raw_reservations.head()

,FO_ROW_ID,ResTk,ResWBS,EVENT_ROW_ID,USER_ID,DATE_TIME_STAMP
0,74,4155748,32414/32513,943475937,alcott43,10/24/2022 9:54
1,74,NaN,NaN,943902421,alcott43,10/25/2022 15:42
2,79,4155668,30990/32063,942950523,kramac15,10/21/2022 15:54
3,79,NaN,NaN,942961454,thukka30,10/21/2022 16:30
4,79,4157252,30990/32063,943831996,jagalu79,10/25/2022 10:55


In [6]:
Fab300_raw_reservations = pd.read_csv("C:\\Users\\fpicaso\\Repos\\Teo-2022-10\\20221108Fab300_raw_reservations.csv")

columns = [
    "FO_ROW_ID",
    "DATE_TIME_STAMP_Begin",
    "DATE_TIME_STAMP_End",
    'EVENT_ROW_ID_Begin', 
    'EVENT_ROW_ID_End',
    'ResTk',
    'USER_ID_Start',
    'USER_ID_End',
    'WBS'
]

for_row_ids = Fab300_raw_reservations["FO_ROW_ID"].unique()

Tools_with_reservations = pd.DataFrame(columns=columns)

for row_id in for_row_ids:
    grpdata = Fab300_raw_reservations[Fab300_raw_reservations["FO_ROW_ID"] == row_id]
    
    df = identify_reservations(grpdata)
    if df.shape[0] > 0:
        print(row_id)
        print("------------------------------")
        print(df.head(5))
        Tools_with_reservations = pd.concat([Tools_with_reservations,df])


Tools_with_reservations

736
------------------------------
ResProperty_1  FO_ROW_ID    ResTk          WBS EVENT_ROW_ID_Begin  \
2                    736  4152709  31499/00880          942682074   
3                    736  4155938  31499/00880          943409222   
4                    736  4155938  31499/00880          943410012   
2                    736  4152709  31499/00880          942682074   
3                    736  4155938  31499/00880          943409222   

ResProperty_1 EVENT_ROW_ID_End DATE_TIME_STAMP_Begin DATE_TIME_STAMP_End  \
2                    943409217   2022-10-20 18:19:00 2022-10-23 19:07:00   
3                    943409962   2022-10-23 19:08:00 2022-10-23 19:12:00   
4                    943471163   2022-10-23 19:12:00 2022-10-24 09:24:00   
2                    943409217   2022-10-20 18:19:00 2022-10-23 19:07:00   
3                    943409962   2022-10-23 19:08:00 2022-10-23 19:12:00   

ResProperty_1 USER_ID_Start USER_ID_End  
2                  groven85       stass  
3        

12107
------------------------------
ResProperty_1  FO_ROW_ID    ResTk          WBS EVENT_ROW_ID_Begin  \
1                  12107  4155179  36003/01224          942536288   
2                  12107  4155604  36003/01224          942862177   
3                  12107      N/a  36003/01224          943866617   
4                  12107  4162018  31199/01873          944175354   
5                  12107  4161512  31199/01873          944372590   

ResProperty_1 EVENT_ROW_ID_End DATE_TIME_STAMP_Begin DATE_TIME_STAMP_End  \
1                    942640665   2022-10-20 10:20:00 2022-10-20 16:41:00   
2                    942952031   2022-10-21 11:07:00 2022-10-21 15:57:00   
3                    943918805   2022-10-25 13:56:00 2022-10-25 16:36:00   
4                    944372589   2022-10-26 13:00:00 2022-10-27 04:11:00   
5                    944748980   2022-10-27 04:11:00 2022-10-28 08:46:00   

ResProperty_1 USER_ID_Start USER_ID_End  
1                  vbelle62    vbelle62  
2      

,FO_ROW_ID,DATE_TIME_STAMP_Begin,DATE_TIME_STAMP_End,EVENT_ROW_ID_Begin,EVENT_ROW_ID_End,ResTk,USER_ID_Start,USER_ID_End,WBS
2,736,2022-10-20 18:19:00,2022-10-23 19:07:00,942682074,943409217,4152709,groven85,stass,31499/00880
3,736,2022-10-23 19:08:00,2022-10-23 19:12:00,943409222,943409962,4155938,stass,stass,31499/00880
4,736,2022-10-23 19:12:00,2022-10-24 09:24:00,943410012,943471163,4155938,stass,groven85,31499/00880
2,736,2022-10-20 18:19:00,2022-10-23 19:07:00,942682074,943409217,4152709,groven85,stass,31499/00880
3,736,2022-10-23 19:08:00,2022-10-23 19:12:00,943409222,943409962,4155938,stass,stass,31499/00880
...,...,...,...,...,...,...,...,...,...
2,24322,2022-10-23 07:47:00,2022-10-23 08:20:00,943317615,943323889,4155975,carola29,carola29,42199/00160
3,24322,2022-10-24 12:21:00,2022-10-24 17:26:00,943520298,943595945,4155735,drijboom,drijboom,42199/00160
4,24322,2022-10-24 17:26:00,2022-10-26 16:50:00,943595947,944236288,4155735,drijboom,scheer40,42199/00160
5,24322,2022-10-27 10:14:00,2022-10-28 09:47:00,944427413,944760571,4157501,drijboom,drijboom,42199/00160


In [7]:
Tools_Parents = pd.read_csv("Tools_Parents.csv")
Tools_Parents["CSIM_TIMESTAMP"] = pd.to_datetime(Tools_Parents["CSIM_TIMESTAMP"])
Tools_Parents

,DEL_FLAG,FACILITY,ENT_NAME,ROW_ID,CSIM_TIMESTAMP,Area
0,N,PLINE300,AIXTRON,74,2022-10-19 11:03:00,Area/DDEPI/Deposition/EPI
1,N,PLINE200,ARCHER,75,2022-10-20 17:45:00,Area/ILIME/Metrology/Inspections
2,N,PLINE300,DNS,76,2022-10-20 21:57:00,Area/WMCE/Clean/Clean
3,N,PLINE300,DUMMY300,78,2022-09-10 11:33:00,Area/Dummy-Area/Dummy-SubArea/Dummy-EntityGroup
4,N,PLINE300,F5_SCD,79,2022-10-20 20:51:00,Area/ILIME/Metrology/Ellipsometry
...,...,...,...,...,...,...
389,N,PLINE200,CHAMEO_UP,25772,2022-07-14 16:52:00,Area/WBA/Assembly/FlipChip
390,N,PLINE300,GT2000,26118,2022-10-14 11:38:00,Area/ILIME/Metrology/Inspections
391,N,PLINE200,UV_MANUAL,26135,2022-10-20 14:07:00,Area/WBA/Assembly/Dicing
392,N,PLINE200,A412_2,26167,2022-10-16 06:25:00,Area/DDDIFFUSION/Anneal/AnnealOven


In [8]:
#FAB300_with_tool_names
Expanded_Tools_parents = pd.merge(
    Tools_with_reservations, 
    Tools_Parents, 
    left_on=["FO_ROW_ID"], 
    right_on=["ROW_ID"], 
    how="left",
    suffixes=["","_y"]
)
Expanded_Tools_parents = Expanded_Tools_parents.rename(
    columns={
        "ENT_NAME":"Tool",
        "USER_ID_Start":"USER_ID_Begin",
        "USER_ID_End":"USER_ID_End"
    }
)
Expanded_Tools_parents_filteredRows = Expanded_Tools_parents[
    Expanded_Tools_parents["USER_ID_Begin"] == Expanded_Tools_parents["USER_ID_End"]
]
Expanded_Tools_parents_filteredRows = Expanded_Tools_parents_filteredRows.rename(
    columns={
        "USER_ID_Begin":"User_Id",
        "DATE_TIME_STAMP_Begin":"Begin",
        "DATE_TIME_STAMP_End":"End"
    }
)
Expanded_Tools_parents_filteredRows = Expanded_Tools_parents_filteredRows.sort_values(["FO_ROW_ID","EVENT_ROW_ID_Begin"])
Fab300_Res_id = range(0,len(Expanded_Tools_parents_filteredRows))
Expanded_Tools_parents_filteredRows["Fab300_Res_id"] = Fab300_Res_id
Expanded_Tools_parents_filteredRows = Expanded_Tools_parents_filteredRows[
    (
        Expanded_Tools_parents_filteredRows["Tool"].notnull() & 
        Expanded_Tools_parents_filteredRows['Tool'].str.len() > 0)
]
columns = [
    "FO_ROW_ID",
    "EVENT_ROW_ID_Begin",
    "Begin",
    "End",
    "Fab300_Res_id",
    "FACILITY",
    "ResTk",
    "Tool",
    "User_Id",
    "WBS"
]
Expanded_Tools_parents_filteredRows[columns]
FAB300_with_tool_names = Expanded_Tools_parents_filteredRows[columns]

#Sample for FO_ROW_ID == 76
FAB300_with_tool_names[FAB300_with_tool_names["FO_ROW_ID"] == 76]

,FO_ROW_ID,EVENT_ROW_ID_Begin,Begin,End,Fab300_Res_id,FACILITY,ResTk,Tool,User_Id,WBS


In [9]:
#IIO_raw_reservations

df = pd.read_csv("IIO_raw_reservations - test.csv").reset_index()
df["Begin"] = pd.to_datetime(df["Begin"],format="%d/%m/%Y %H:%M")
df["End"] = pd.to_datetime(df["End"],format="%d/%m/%Y %H:%M")
df = df.sort_values(["WBS","Tool","FACILITY","Module","Begin"])
df["End_Down"] = df["End"].shift(1)
df["Adjacent_Down"] = np.where(df['Begin']==df['End_Down'], True, False)
df["IndexCopy"] = np.where(df["Adjacent_Down"] == True,np.nan,df["index"])
df["IndexCopy2"] = df["IndexCopy"].fillna(method='ffill')
df["Module"] = df["Module"].fillna("")
df["Description"] = df["Description"].fillna("")
df = df[['WBS','FACILITY', 'Module','Tool','Begin', 'Description', 'End', 'User_id','IndexCopy2','End_Down']]


params = {
    'Begin': 'min',
    'End': 'max',
    'Description': lambda x: ';'.join(sorted(pd.Series.unique(x))),
    'User_id': lambda x: ';'.join(sorted(pd.Series.unique(x)))
}
sub = df[["WBS","Tool","FACILITY","Module","Begin","End","Description",'User_id','IndexCopy2']]
GroupedRows1 = sub.groupby(["WBS","Tool","FACILITY","Module",'IndexCopy2']).agg(params).reset_index()

params = {
    'Module': lambda x: ';'.join(sorted(pd.Series.unique(x))),
    'Description': 'first',
    'User_id': 'first'
}


GroupedRows1 = GroupedRows1.groupby(["WBS","Tool","FACILITY","Begin", "End"]).agg(params).reset_index()
GroupedRows1 = GroupedRows1.sort_values(["FACILITY", "Tool", "WBS", "Begin", "End"])

Index = range(0,len(GroupedRows1))
GroupedRows1["IIO_Res_id"] = Index
IIO_without_modules = GroupedRows1.copy()
IIO_without_modules

,WBS,Tool,FACILITY,Begin,End,Module,Description,User_id,IIO_Res_id
291,31012/01506,300MM_DICER,PLINE200,2022-09-29 09:00:00,2022-09-29 17:00:00,,lots on hold,schoofs,0
292,31012/01506,300MM_DICER,PLINE200,2022-10-14 13:30:00,2022-10-14 17:00:00,,lots on hold,schoofs,1
293,31012/01506,300MM_DICER,PLINE200,2022-10-18 13:30:00,2022-10-18 17:00:00,,lots on hold,schoofs,2
2499,36671/01224,300MM_DICER,PLINE200,2022-10-05 11:00:00,2022-10-05 12:00:00,,IN4MED lot,ameghn57,3
2500,36671/01224,300MM_DICER,PLINE200,2022-10-07 09:30:00,2022-10-07 12:00:00,,DEVINP LOTS,ameghn57,4
...,...,...,...,...,...,...,...,...,...
2356,36003/02120,ZETA,PLINE300,2022-09-30 13:30:00,2022-09-30 17:00:00,,Recipe setting,tuncaa41,3097
2357,36003/02120,ZETA,PLINE300,2022-10-07 09:30:00,2022-10-07 11:00:00,,Recipe practice,tuncaa41,3098
2358,36003/02120,ZETA,PLINE300,2022-10-17 12:30:00,2022-10-17 17:00:00,,Recipe setting,tuncaa41,3099
2359,36003/02120,ZETA,PLINE300,2022-10-24 10:00:00,2022-10-24 12:00:00,,recipe testing,tuncaa41,3100
